In [159]:
import numpy as np
import pandas as pd
import os
from sklearn import model_selection, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import regex as re
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from string import *
from sklearn.ensemble import RandomForestClassifier

In [160]:
pwd = os.getcwd()
pwd

'C:\\Users\\my\\csv_files_cairn'

In [3]:
os.chdir('csv_files_cairn')

In [161]:
data = pd.read_excel('Cairn_sample_dataset.xlsx')

In [162]:
data.shape

(5060, 16)

In [142]:
# data.dtypes

In [7]:
data[:3]

,Company,Project,Event code,Common Well Name,Report Date,From,To,Hrs (hr),Phase,Class,Code,Code description,Sub,code + sub,24 Hr Operation Summary,Operation
0,Cairn India Ltd.,RJ-ON-90-1,ABN,Tukaram-SE-A,2019-03-02 00:00:00,2019-03-02 07:00:00,2019-03-02 19:00:00,12.00,ABAND,U,24.0,NaN,NaN,24,Waiting on CT,Waiting on CT
1,Cairn India Ltd.,RJ-ON-90-1,ABN,Tukaram-SE-A,2019-03-03 00:00:00,2019-03-03 07:00:00,2019-03-03 23:17:00,16.28,ABAND,U,24.0,NaN,NaN,24,Waiting on CT,Waiting on CT
2,Cairn India Ltd.,RJ-ON-90-1,ABN,Tukaram-SE-A,2019-03-04 00:00:00,2019-03-04 07:00:00,2019-03-04 09:00:00,2.00,ABAND,U,24.0,NaN,NaN,24,Waiting on CT,Waiting on CT


In [8]:
dataset= data.loc[:, ['Operation', 'code + sub']]

In [9]:
dataset = dataset.rename(columns={'code + sub': 'Code'})

In [10]:
dataset.shape

(5060, 2)

In [11]:
dataset[3025:3029]

,Operation,Code
3025,Night crew Arrived on Location handed over.,NaN
3026,"Conducted TBT, discussed Job program (especial...",33A
3027,Waiting for area to be clear from Rig equipmen...,24H
3028,Waiting well hand over from production.,24A


In [12]:
dataset.isnull().sum()

Operation     12
Code         861
dtype: int64

In [13]:
dataset.dropna(inplace=True)

In [14]:
dataset_copy = dataset.copy()

In [15]:
#Converting the feature to lower case
# dataset['Operation'] = dataset['Operation'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dataset['Operation'] = dataset['Operation'].apply(lambda x:x.lower())

#Removing punctuations
# dataset['Operation'] = dataset['Operation'].str.replace('[^\w\s]','')
dataset['Operation'] = dataset['Operation'].str.translate(str.maketrans('', '', punctuation))

#Removing digits
dataset['Operation'] = dataset['Operation'].str.translate(str.maketrans('', '', digits))

#Removing stop words
stop = stopwords.words('english')
dataset['Operation'] = dataset['Operation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


In [16]:
#Full dataset contains 147 classes

dataset['Code'].nunique()

147

## Event code: CTU - Modeling with all sub-codes

In [17]:
dataset_CTU = data.loc[data['Event code']=='CTU']

In [18]:
dataset_CTU['code + sub'].nunique()

34

In [19]:
dataset_CTU= dataset_CTU.loc[:, ['Operation', 'code + sub']]

In [20]:
dataset_CTU = dataset_CTU.rename(columns={'code + sub': 'Code'})

In [21]:
dataset_CTU.dropna(inplace=True)

In [22]:
# dataset_CTU['Code'].value_counts()

In [23]:
dataset_CTU.shape

(567, 2)

In [24]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(dataset_CTU['Operation'], dataset_CTU['Code'], test_size = 0.3, random_state = 42)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
actual = y_test

tfidf_vect = TfidfVectorizer() 
tfidf_vect.fit(dataset_CTU['Operation'])
x_train_tf = tfidf_vect.transform(x_train)
x_test_tf = tfidf_vect.transform(x_test)

def model_utility(classifier, x_train, y_train, x_test):
    classifier.fit(x_train, y_train)
    pred = classifier.predict(x_test)
    train_acc = classifier.predict(x_train)
    x = metrics.accuracy_score(actual, pred)
    y = metrics.accuracy_score(y_train,train_acc)
    return print('Training Acc:',y, "\nTest Accuracy:",x)

print("SVM")
svm_model = model_utility(svm.SVC(), x_train_tf, y_train, x_test_tf)

print("\nNB")
mnb_model = model_utility(MultinomialNB(), x_train_tf, y_train, x_test_tf)

print("\nRF")
rf_model = model_utility(RandomForestClassifier(), x_train_tf, y_train, x_test_tf)


SVM
Training Acc: 0.2398989898989899 
Test Accuracy: 0.1111111111111111

NB
Training Acc: 0.5075757575757576 
Test Accuracy: 0.09941520467836257

RF
Training Acc: 0.9772727272727273 
Test Accuracy: 0.0935672514619883


## Event code: CTU - Modeling with top 5 sub-codes

In [25]:
dataset_CTU_top = data.loc[data['Event code']=='CTU']

dataset_CTU_top['Code_new'] = np.where(dataset_CTU_top['code + sub'].isin(['87C', '87F', '87O', '88A', '86A']), dataset_CTU_top['code + sub'], 'Others')

#Converting the feature to lower case
# dataset['Operation'] = dataset['Operation'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dataset_CTU_top['Operation'] = dataset_CTU_top['Operation'].apply(lambda x:x.lower())

#Removing punctuations
# dataset['Operation'] = dataset['Operation'].str.replace('[^\w\s]','')
dataset_CTU_top['Operation'] = dataset_CTU_top['Operation'].str.translate(str.maketrans('', '', punctuation))

#Removing digits
dataset_CTU_top['Operation'] = dataset_CTU_top['Operation'].str.translate(str.maketrans('', '', digits))

#Removing stop words
stop = stopwords.words('english')
dataset_CTU_top['Operation'] = dataset_CTU_top['Operation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

x_train, x_test, y_train, y_test = model_selection.train_test_split(dataset_CTU_top['Operation'], dataset_CTU_top['Code_new'], test_size = 0.3, random_state = 42)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
actual = y_test

tfidf_vect = TfidfVectorizer() 
tfidf_vect.fit(dataset_CTU['Operation'])
x_train_tf = tfidf_vect.transform(x_train)
x_test_tf = tfidf_vect.transform(x_test)

#Utility function for multiple models

def model_utility(classifier, x_train, y_train, x_test):
    classifier.fit(x_train, y_train)
    pred = classifier.predict(x_test)
    train_acc = classifier.predict(x_train)
    x = metrics.accuracy_score(actual, pred)
    y = metrics.accuracy_score(y_train,train_acc)
    return print('Training Acc:',y, "\nTest Accuracy:",x)

print("SVM")
svm_model = model_utility(svm.SVC(), x_train_tf, y_train, x_test_tf)

print("\nNB")
mnb_model = model_utility(MultinomialNB(), x_train_tf, y_train, x_test_tf)

print("\nRF")
rf_model = model_utility(RandomForestClassifier(), x_train_tf, y_train, x_test_tf)


SVM
Training Acc: 0.4020100502512563 
Test Accuracy: 0.3567251461988304

NB
Training Acc: 0.7864321608040201 
Test Accuracy: 0.7426900584795322

RF
Training Acc: 0.9773869346733668 
Test Accuracy: 0.7485380116959064


C:\Users\my\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\my\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\my\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

## To calculating accuracy at each sub-code level

In [26]:
encoder.transform(['86A', '87C', '87F', '87O', '88A', 'Others'])

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [27]:
nb1 = RandomForestClassifier()
nb1.fit(x_train_tf, y_train)
train_pred = nb1.predict(x_train_tf) 
test_pred = nb1.predict(x_test_tf)
train_accuracy = print('Training accuracy of NB: ', metrics.accuracy_score(y_train, train_pred))
test_accuracy = print('Test accuracy of NB: ', metrics.accuracy_score(y_test, test_pred))


Training accuracy of NB:  0.9798994974874372
Test accuracy of NB:  0.7134502923976608


In [28]:
print('Test and Pred count: ', len(y_test.tolist()), len(test_pred.tolist()))

Test and Pred count:  171 171


In [29]:
print('Test and pred values', set(y_test.tolist()), set(test_pred.tolist()))

Test and pred values {0, 1, 2, 3, 4, 5} {0, 1, 2, 3, 4, 5}


In [30]:
print('Train and Pred count: ', len(y_train.tolist()), len(train_pred.tolist()))

Train and Pred count:  398 398


In [31]:
print('Train and Pred values: ', set(y_train.tolist()), set(train_pred.tolist()))

Train and Pred values:  {0, 1, 2, 3, 4, 5} {0, 1, 2, 3, 4, 5}


In [32]:
test_tuple = list(zip(y_test.tolist(), test_pred.tolist()))

In [33]:
test_df = pd.DataFrame(test_tuple, columns=['Test_Actual', 'Test_Pred'])

In [34]:
train_tuple = list(zip(y_train.tolist(), train_pred.tolist()))

In [35]:
train_df = pd.DataFrame(train_tuple, columns=['Train_Actual', 'Train_Pred'])

In [36]:
# test_df.to_csv('svm_test.csv')

In [37]:
# train_df.to_csv('svm_train.csv')

In [38]:
# pd.DataFrame(list(zip(x_train, y_train, train_pred)), columns=['Train', 'Actual', 'Pred']).to_csv('Train_act_pred.csv')

## Oct'18 till Mar'19 data - 1st to 20th of each month

In [39]:
data_6m = pd.read_excel('Cairn_6m.xlsx')

In [40]:
data_6m[:3]

,Company,Project,Event code,Common Well Name,Report Date,From,To,Hrs (hr),Phase,Class,Code,Sub,24 Hr Operation Summary,Operation
0,Cairn India Ltd.,RJ-ON-90-1,ELN,MANG-244-11-101,2018-10-01,2018-10-01,2018-10-01 03:00:00,3.0,WSRVS,P,87,U,COmpleted C/O passes. RD unit.,RST C/O Pass-3 recorded from 1190 m - 1065 m a...
1,Cairn India Ltd.,RJ-ON-90-1,PMP,AISH-057-04-211,2018-10-01,2018-10-01,2018-10-01 07:30:00,7.5,WSRVS,P,24,D,NaN,Stand By on fluid .
2,Cairn India Ltd.,RJ-ON-90-1,MWS,MANG-197-01-209,2018-10-01,2018-10-01,2018-10-01 05:00:00,5.0,PRDTN,P,86,A,Continue R/ U; Commenced P/ T;,Rigged up flow lines from 5 way diverter manif...


In [41]:
data_6m.dtypes

Company                            object
Project                            object
Event code                         object
Common Well Name                   object
Report Date                datetime64[ns]
From                       datetime64[ns]
To                         datetime64[ns]
Hrs (hr)                          float64
Phase                              object
Class                              object
Code                               object
Sub                                object
24 Hr Operation Summary            object
Operation                          object
dtype: object

In [42]:
data_6m_87 = data_6m.loc[(data_6m['Code']==87) & (((data_6m['Event code']=='CTU') & (data_6m['Sub'].isin(['R', 'C', 'O', 'B', 'F']))) |
                                                  ((data_6m['Event code']=='ELN') & (data_6m['Sub'].isin(['S', 'O', 'U', 'Z', 'X']))) |
                                                  ((data_6m['Event code']=='MWS') & (data_6m['Sub'].isin(['S', 'R', 'C', 'O', 'F']))) |
                                                  ((data_6m['Event code']=='SLN') & (data_6m['Sub'].isin(['R', 'C', 'B', 'Y', 'P']))) |
                                                  ((data_6m['Event code']=='PMP') & (data_6m['Sub'].isin(['B'])))
                                                  ), :]

In [43]:
data_6m_87.shape

(6942, 14)

In [44]:
data_6m_87.isnull().sum()

Company                      0
Project                      0
Event code                   0
Common Well Name             0
Report Date                  0
From                         1
To                           1
Hrs (hr)                    58
Phase                      198
Class                      159
Code                         0
Sub                          0
24 Hr Operation Summary    312
Operation                    0
dtype: int64

In [45]:
data_6m_87['Code_Sub'] = data_6m_87['Code'].map(str) + data_6m_87['Sub'].map(str)

C:\Users\my\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
data_6m_87_mod = data_6m_87.loc[:, ['Event code', 'Operation', 'Code_Sub']]

In [47]:
data_6m_87_mod = data_6m_87_mod.reset_index(drop=True)

In [48]:
data_6m_87_mod[:3]

,Event code,Operation,Code_Sub
0,ELN,RST C/O Pass-3 recorded from 1190 m - 1065 m a...,87U
1,SLN,Made up T/s with IRBP P/T. and stab lubricator...,87P
2,MWS,CT Start RIH while pumping nitrified fluid. At...,87C


In [49]:
len(data_6m_87_mod)

6942

In [50]:
data_6m_87_mod1 = data_6m_87_mod.copy()

In [106]:
words_map = pd.read_csv('words_map.csv')
words_map.head()

,Event code,Code_Sub,Words_to_match
0,CTU,87R,"['RIH', 'POOH']"
1,CTU,87C,"['Cleanout', 'Jetting']"
2,CTU,87O,['Others']
3,CTU,87B,['Bullheading']
4,CTU,87F,['Flowback (N2)']


In [134]:
from ast import literal_eval

def check_string(row):
    a = literal_eval(row['Words_to_match'])
    b = row['Operation']
    c = row['Code_Sub']
    
    def t_or_f(a,b):
        if any(x in b for x in a):
            return(True)
        else:
            return(False)
    
    match = t_or_f(a,b)
    match = np.where(match is True,c,'')
    
    return(match)

In [118]:
# data_6m_87_mod1.head()
words_map.head()

,Event code,Code_Sub,Words_to_match
0,CTU,87R,"['RIH', 'POOH']"
1,CTU,87C,"['Cleanout', 'Jetting']"
2,CTU,87O,['Others']
3,CTU,87B,['Bullheading']
4,CTU,87F,['Flowback (N2)']


In [130]:
output = data_6m_87_mod1.merge(words_map, on=['Event code', 'Code_Sub'], how = 'left')

In [132]:
output['Words_to_match'] = np.where(output['Words_to_match'].isnull(),"['####']",output['Words_to_match'])

In [135]:
output.head()

,Event code,Operation,Code_Sub,Words_to_match
0,ELN,RST C/O Pass-3 recorded from 1190 m - 1065 m a...,87U,['####']
1,SLN,Made up T/s with IRBP P/T. and stab lubricator...,87P,"['Plug Setting', 'Retrieval']"
2,MWS,CT Start RIH while pumping nitrified fluid. At...,87C,['Cleanout']
3,SLN,"Made up T/s with 2.0"" JUC P/T. and stab lubric...",87P,"['Plug Setting', 'Retrieval']"
4,SLN,"Made up T/s with 2.0"" RB P/T. and stab lubrica...",87P,"['Plug Setting', 'Retrieval']"


In [136]:
output['Flag'] = output.apply(check_string, axis=1)

In [137]:
output.head()

,Event code,Operation,Code_Sub,Words_to_match,Flag
0,ELN,RST C/O Pass-3 recorded from 1190 m - 1065 m a...,87U,['####'],
1,SLN,Made up T/s with IRBP P/T. and stab lubricator...,87P,"['Plug Setting', 'Retrieval']",
2,MWS,CT Start RIH while pumping nitrified fluid. At...,87C,['Cleanout'],
3,SLN,"Made up T/s with 2.0"" JUC P/T. and stab lubric...",87P,"['Plug Setting', 'Retrieval']",
4,SLN,"Made up T/s with 2.0"" RB P/T. and stab lubrica...",87P,"['Plug Setting', 'Retrieval']",


In [140]:
output.to_excel('output.xlsx')